# Skapa vmodel och model på senaste datat utan test

In [1]:
import pandas as pd
from datetime import datetime as dt
import yfinance as yf
import matplotlib.pyplot as plt 
from catboost import CatBoostRegressor,Pool,utils
import time
plt.style.use('fivethirtyeight')
pd.set_option('display.width',200)

In [2]:
def set_kolumner():
    rkolumner = ['vol_y', 'ret_2', 'vol_2', 'ret_3', 'vol_3', 'ret_4', 'vol_4', 'ret_5', 'vol_5', 
                'ret_6', 'vol_6', 'ret_7', 'vol_7', 'ret_8', 'vol_8', 'ret_9', 'vol_9', 'ret_10', 
                'vol_10', 'year', 'month', 'wday', 'day']
    vkolumner = rkolumner[1:]
    
    return vkolumner, rkolumner

# let the data include vol_y as input to ret_y-training

In [3]:
def fit_data(train, iterations=550):
    
    vkolumner, rkolumner = set_kolumner()
    
    ret_y=train.ret_y
    vol_y=train.vol_y

    print('shape train av ret_y',train[rkolumner].shape)   
    print('shape train av vol_y',train[vkolumner].shape)   
    
    vmodel = CatBoostRegressor(iterations=iterations,
        verbose=False, random_state=2021)
    
    vmodel.fit(train[vkolumner],y=vol_y,use_best_model=False) # train on 'kolumner' except vol_y and ret_y
    
    model = CatBoostRegressor(iterations=iterations,
        verbose=False, random_state=2021)
    
    model.fit(train[rkolumner],y=ret_y,use_best_model=False) # includes vol_y for training
    
    return vmodel, model

In [ ]:
def full_model( data):
    data = data.tail(720)  # använd de 720 sista dagarna (2 år)
    vmodel, model = fit_data(data)
    print(data.shape)
    return vmodel,model

def pct_change(df, ny, gammal, dagar):
    df[ny+'_'+str(dagar)] = df[gammal].pct_change(dagar)

# Historik för proc utveckling av 'Adj Close' och volume
def add_history(df):
    for i in range (10):
        pct_change(df,'ret','Adj Close',i+1)
        pct_change(df,'vol','Volume',i+1)
    
    df.rename(columns={'vol_1':'vol_y', 'ret_1':'ret_y'},inplace=True)
    df = df.dropna()
    return df

# kolumner för år, månad, dag samt predict
def add_more_cols(df):
    df['pred_Volume'] = None
    df['pred_AdjClose'] = None
        
    df['year']=pd.DatetimeIndex(df.index).year
    df['month']=pd.DatetimeIndex(df.index).month
    df['wday']=pd.DatetimeIndex(df.index).weekday
    df['day']=pd.DatetimeIndex(df.index).day
   
    return df


In [ ]:
tickers = ['ETH-USD','BTC-USD','BCH-USD','ZRX-USD','XRP-USD',]

for ticker in tickers:
    print()
    print(ticker)
    df = yf.download([ticker], progress=False)[['Adj Close','Volume']]
    df = df.iloc[-725:]
    df=add_history(df)
    df=add_more_cols(df)
    # print(df.shape)

    vmodel, model = full_model(df)
    vmodel.save_model(f'{ticker[:3]}_vmodel')
    model.save_model(f'{ticker[:3]}_model')
